In [1]:
import dotenv

dotenv.load_dotenv()

True

## Scrape Search Results from Bright Data

In [20]:
import urllib.request
import urllib.parse
import pandas as pd
import json
import os

query = pd.read_csv('query.csv').head(2)
for index, row in query.iterrows():
    if row['query'] != 'initial' or pd.isnull(row['result']):
        search = str(row['query'])

        opener = urllib.request.build_opener(
            urllib.request.ProxyHandler(
                {'http': os.environ['BRIGHTDATA_SERP_KEY'],
                'https': os.environ['BRIGHTDATA_SERP_KEY']}))
        search_query = urllib.parse.quote_plus(search)

        results = json.loads(opener.open(f'http://www.google.com/search?q={search_query}&brd_json=1').read())
        # need to format json result better
        # print(json.dumps(results, indent=2))
        query.at[index, 'result'] = json.dumps(results)
    else: pass

HTTPError: HTTP Error 407: Auth Failed (code: ip_forbidden)

In [3]:
query.to_csv('query_updated.csv', index=False)

In [2]:
import pandas as pd
import json

query = pd.read_csv('query_updated.csv')

def format_result_for_ai(result: str):
    alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    result = json.loads(result)
    human_readable = [
        f"{alphabet[i]}. {r['title']} ({r['display_link']}) - {r['description']}"
        for i, r in enumerate(result['organic'])
    ]
    return "\n".join(human_readable)

print(format_result_for_ai(query.iloc[1]['result']))

A. Wind site seeking assessed value cut | News | nny360.com (https://www.nny360.com › news › wind-site-seeking-ass...) - Sep 11, 2018 — LOWVILLE — The owners of Maple Ridge Wind Farm are seeking to have the assessments on the project's Lewis County properties reduced by ...
B. Two controversial north country wind projects 'no longer ... (https://www.nny360.com › news › lewiscounty › two-co...) - Dec 13, 2020 — Two controversial north country wind projects 'no longer seeking' Article 10 approval | Lewis County | nny360.com.
C. Wind Farm Buffets Family, Town Relations (https://www.npr.org › 2008/04/09 › wind-farm-buffets-f...) - Apr 9, 2008 — "It has caused friction, family against family or neighbor against neighbor," says Arleigh Rice, town supervisor for Lowville, one of the three ...
D. Windmills split town and families (https://www.nbcnews.com › wbna26242604) - Aug 16, 2008 — The $400 million Maple Ridge wind project, the largest in New York state, brought money and jobs and a wondr

## Query OpenAI

In [2]:
import json
search_results = json.loads(row['result'])
search_results

NameError: name 'row' is not defined

In [5]:
query.iloc[1]

query     Maple Ridge Wind Farm New York Lewis County (c...
gpt       [{"score": 8, "description": " This article di...
human                                                   NaN
result    {"general": {"search_engine": "google", "resul...
Name: 1, dtype: object

In [16]:
system_prompt = f'Our aim is to take a set of search results from Google corresponding to the following search query: {query.iloc[1]["query"]} and determine which search results are relevant to our research question. Based on the description of each URL and other metadata, we would like to identify which search results are most relevant to this research question: "What is the narrative surrounding the development of this renewable energy project in this location, and what evidence of opposition or support for the project can be identified?" Score each search result with a number between 1-5, with 1 being the least relevant and 5 being the most relevant. Here are examples of what might receive the following scores:\n1 - an article that might be related to renewable energy but does not have any mention of the project or location in question\n2 - an article that might be related to renewable energy near the location in question but does not mention the project\n3 - an article that mentions the project and location in question, but only provides basic information about the project and no information on opposition or support\n5 - an article that describes the narrative of the project development in detail, including mentions of opposition and support. Note the search results are listed in order of apperance from Google.Respond in the format:\nA=4 (< 8 word justification)\nB=2 (< 8 word justification)\n...'


import openai
import dotenv
import os

dotenv.load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG")

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo", 
  messages = [{"role": "system", "content" : system_prompt},
              {"role": "user", "content" : format_result_for_ai(query.iloc[1]['result'])}]
)

In [ ]:
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT

anthropic = Anthropic()
completion = anthropic.completions.create(
    model="claude-2",
    max_tokens_to_sample=300,
    prompt=f"{HUMAN_PROMPT} How many toes do dogs have?{AI_PROMPT}",
)
print(completion.completion)


In [17]:
print(completion.choices[0].message.content)

A = 3 (Provides basic information about the wind farm, but no information on opposition or support)
B = 3 (Provides basic information about wind projects in the area, but no information on opposition or support)
C = 4 (Describes the friction and conflicts caused by the wind farm)
D = 5 (Describes the positive impacts and prosperity brought by the Maple Ridge wind project, as well as the friction caused)
E = 3 (Provides basic information about Maple Ridge Wind Farm, but no information on opposition or support)
F = 5 (Describes the conflicts and lawsuits surrounding the wind farm)
G = 2 (Does not provide relevant information about opposition or support)
H = 4 (Discusses the opposition to wind farms in general, but not specifically about Maple Ridge)
I = 4 (Describes the positive impact of the wind farm on local governments, but no information on opposition or support)
J = 3 (Provides basic information about Maple Ridge Wind Farm, but no information on opposition or support)


In [15]:
print(format_result_for_ai(query.iloc[1]['result']))

A. Wind site seeking assessed value cut | News | nny360.com (https://www.nny360.com › news › wind-site-seeking-ass...) - Sep 11, 2018 — LOWVILLE — The owners of Maple Ridge Wind Farm are seeking to have the assessments on the project's Lewis County properties reduced by ...
B. Two controversial north country wind projects 'no longer ... (https://www.nny360.com › news › lewiscounty › two-co...) - Dec 13, 2020 — Two controversial north country wind projects 'no longer seeking' Article 10 approval | Lewis County | nny360.com.
C. Wind Farm Buffets Family, Town Relations (https://www.npr.org › 2008/04/09 › wind-farm-buffets-f...) - Apr 9, 2008 — "It has caused friction, family against family or neighbor against neighbor," says Arleigh Rice, town supervisor for Lowville, one of the three ...
D. Windmills split town and families (https://www.nbcnews.com › wbna26242604) - Aug 16, 2008 — The $400 million Maple Ridge wind project, the largest in New York state, brought money and jobs and a wondr